In [1]:
from keras_model_manager import Load
from FashionNet import FashionNet
from keras.preprocessing.image import img_to_array
import imutils
import numpy as np
import cv2

Using TensorFlow backend.
/Users/aaronwong/miniconda3/envs/kerasenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/aaronwong/miniconda3/envs/kerasenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/aaronwong/miniconda3/envs/kerasenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

### Load Classifier Labels

In [2]:
label_paths = [
    'clothes-and-color-multi-categories_labels.csv', 
    'clothes-and-color-multi-colors_labels.csv'
]
classifiers = []
for l in label_paths:
    classifiers.append(Load.load_classes_from_csv(l))

In [3]:
print(classifiers)
category_classes = classifiers[0][0]
color_classes = classifiers[1][0]

[[['dress', ' jeans', ' shirt', ' shoes']], [['black', ' blue', ' red']]]


### Load Model

In [4]:
model = FashionNet.build((96, 96, 3), 
                         num_categories=len(category_classes), 
                         num_colors=len(color_classes), 
                         final_act="softmax")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Tensor("dropout_3/cond/Merge:0", shape=(?, 8, 8, 128), dtype=float32)


### Load Pre-Trained Weights

In [5]:
model.load_weights("clothes-and-color-multi_weights.h5")

### Create Prediction Method

In [6]:
def predict_on_image(model, image_path):
    # load the image
    image = cv2.imread(image_path)
    output = imutils.resize(image, width=400)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # preprocess the image for classification
    image = cv2.resize(image, (96, 96))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    
    (category_prob, color_prob) = model.predict(image)
    
    # find indexes of both the category and color outputs with the
    # largest probabilities, then determine the corresponding class
    # labels
    category_idx = category_prob[0].argmax()
    color_idx = color_prob[0].argmax()
    category_label = category_classes[category_idx]
    color_label = color_classes[color_idx]
    print("{0} {1}".format(color_label, category_label))
    print("category: {} ({:.2f}%)".format(category_label, category_prob[0][category_idx] * 100))
    print("color: {} ({:.2f}%)".format(color_label, color_prob[0][color_idx] * 100))

### Apply Model to prediction method with path for image

In [7]:
predict_on_image(model, 'multi-output-test-examples/black_yeezy.jpg')

black  shoes
category:  shoes (99.90%)
color: black (88.82%)
